## Vector Store: FAISS, CHROMA

# 1) OllamaLLm and HuggingFaceEmbedding: mistralai/Mistral-7B-Instruct-v04

## Document Loader
    pdf loader : langchain inbuilt document loader 

In [9]:
from langchain_community.document_loaders import PyPDFLoader
file_path = ("Leave_Policy_2024.pdf")
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
len(pages)


4

## Split
    smaller chunks

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
len(splits)


15

## New Embeddings

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


In [7]:
new_embeddings = HuggingFaceEmbeddings(model_name= "sentence-transformers/all-mpnet-base-v2")


c:\Users\30078206\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
pip install faiss-cpu


   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB 165.2 kB/s eta 0:01:29
   ---------------------------------------- 0.0/14.6 MB 217.9 kB/s eta 0:01:07
   ---------------------------------------- 0.0/14.6 MB 245.8 kB/s eta 0:01:00
   ---------------------------------------- 0.1/14.6 MB 383.3 kB/s eta 0:00:38
   ---------------------------------------- 0.2/14.6 MB 656.4 kB/s eta 0:00:22
   ---------------------------------------- 0.2/14.6 MB 618.3 kB/s eta 0:00:24
   ---------------------------------------- 0.2/14.6 MB 618.3 kB/s eta 0:00:24
    --------------------------------------- 0.2/14.6 MB 513.4 kB/s eta 0:00:29
    --------------------------------------- 0.3/14.6 MB 605.3 kB/s eta 0:00:24
    --------------------------------------- 0.3/14.6 MB 723.4 kB/s eta 0:00:20
   - -------------------------------------- 0.4/14.6 MB 782.8 kB/s et

In [18]:
docsearch = FAISS.from_documents(splits, new_embeddings)

In [19]:
###############################3


In [20]:
from langchain.chains.question_answering import load_qa_chain


In [ ]:
gb_chain = load_qa_chain(llm, chain_type="stuff")


## Create vector store
    stores embeddings of Documents

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

In [4]:
## Not local
new_embeddings = HuggingFaceEmbeddings(model_name= "sentence-transformers/all-mpnet-base-v2")
vectorstore = Chroma.from_documents(documents=splits, embedding=new_embeddings)
vectorstore

c:\Users\30078206\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\30078206\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


NameError: name 'Chroma' is not defined

## Create retriever

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity")

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [21]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

### LLM Used:

In [ ]:
# !ollama pull mistral

In [22]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [23]:
gb_chain =  load_qa_chain(llm, chain_type="stuff")

In [24]:
gb_chain

StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=ChatOllama(model='mistral')), document_variable_name='context')

In [25]:
query = "Give me a summary of the document"
docs=docsearch.similarity_search(query=query)
gb_chain.run(input_documents = docs, question = query)

### Creating Prompt

In [ ]:

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)


### New Terms:
- Multiqueryretriever: 
    - automates process of tuning
    - to generate multiple queries from different perspective 
    - for each query- returns relevant documents,, takes union across all 
    - Overcomes the limitation of distance based retrieval


In [ ]:
retriever = MultiQueryRetriever.from_llm(
    retriever, 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
questions = chain.invoke("Make a list of questions from the document")

In [ ]:
import re

In [ ]:
answer =re.sub(r'[0-9]','',questions)
question_list = answer.split('\n')
question_list

In [ ]:
chain.invoke(question_list[0])

In [ ]:
''' Taking too much time'''
# answer_list = []
# for question in question_list:
#     ans = chain.invoke(question)
#     answer_list.append(ans)

In [ ]:
print("ANSWERS-----")
# print(answer_list)

In [ ]:
question_bank = [
    "Give me a summary of the document",
    "Tell me all about the leaves",
    "Make a list of questions from the document"
]

--------------------------------------------(2)--------------------------------------------